In [45]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
import requests

trading = betfairlightweight.APIClient(username="",
                                       password="",
                                       app_key="")
trading.login_interactive()

import nbimporter
import time
import undetected_chromedriver as webdriver
from selenium.webdriver.common.by import By

#startup
driver = webdriver.Chrome()
driver.get("https://www.sportsbet.com.au/")



In [46]:
####SPORTSBET
driver.find_element(By.XPATH, "//button[@data-automation-id='header-login-touchable']").click()
time.sleep(3)
driver.find_element(By.XPATH, "//input[@id='username']").send_keys("")
driver.find_element(By.XPATH, "//input[@id='password']").send_keys("")
time.sleep(1)
driver.find_element(By.XPATH,"//div[@data-automation-id='login-button']").click()

In [ ]:


def place_bet():
    bet_no = int(driver.find_element(By.XPATH, f'//span[@data-automation-id="header-bet-count"]').text)
    
    try:
        #check if betslip open
        driver.find_element(By.XPATH, f'//div[@data-automation-id="betslip-stake-all-singles"]').click()
    
    except:
        #if not open open
        driver.find_element(By.XPATH, f'//span[@data-automation-id="header-bet-count"]').click()
        
    #place bet
    if bet_no == 1:
        driver.find_element(By.XPATH, f'//input[@data-automation-id="betslip-stake"]').send_keys("0.01")
        driver.find_element(By.XPATH, f'//span[@data-automation-id="betslip-footer-right-button-label"]').click()
        driver.find_element(By.XPATH, f'//i[@data-automation-id="betslip-header-hide"]').click()
        
    elif bet_no > 1:
        driver.find_element(By.XPATH, f'//div[@data-automation-id="betslip-stake-all-singles"]//input').send_keys("0.01")
        driver.find_element(By.XPATH, f'//span[@data-automation-id="betslip-footer-right-button-label"]').click()
        driver.find_element(By.XPATH, f'//i[@data-automation-id="betslip-header-hide"]').click()
        
    else:
        print("ERRRORR")
        
## GET NEXT RACE
def sb_get_next_race(next_race_block):
    #selenium get next race  
    driver.find_element(By.XPATH,next_race_block).click()
    url = driver.current_url
    print("next race is :" + url)
    return url[-7:]

def get_sb_selections(next_race_block):
    try:
        race_id = sb_get_next_race(next_race_block)
        print(race_id)

        response = requests.get(f"https://www.sportsbet.com.au/apigw/sportsbook-racing/Sportsbook/Racing/Events/{race_id}/RacecardWithContext?classId=1")
        card_json = json.loads(response.text)
        full_card = card_json.get("racecardEvent")
                        #selections = market[0]["selections"]
                        #print('Next Race Card:', json.dumps(selections, indent=4))
        selections_dict = full_card["markets"][0]["selections"]

        for i in selections:
            if i["statusCode"] == "S":
                selections.pop(selections.index(i))


    #print (json.dumps(selections, indent=4))
    except:
        print("Please check browser")

    
    return selections_dict



In [49]:
selections = get_sb_selections("//div[@data-automation-id='next-races-ribbon-cell-1']")

next race is :https://www.sportsbet.com.au/horse-racing/international/fairyhouse/race-2-6931788
6931788


In [ ]:
while True:
    time.sleep(5)

    
    #### NEXT RACE BETFAIR
    sportsbet_venue = str(driver.find_element(By.XPATH, f'//div[@data-automation-id="contextual-nav-info-container"]//h1').text)
    print(sportsbet_venue)

    betfair_selections = []

    # make event type request to find horse racing event type
    horse_racing_event_type_id = trading.betting.list_event_types(
        filter=filters.market_filter(text_query="Horse Racing")
    )


    # returns one result
    #print(horse_racing_event_type_id)

    for event_type in horse_racing_event_type_id:
        # prints id, name and market count
        print(event_type.event_type.id, event_type.event_type.name, event_type.market_count)
        horse_racing_id = event_type.event_type.id

        # list all horse racing market catalogues
        market_catalogues = trading.betting.list_market_catalogue(
            filter=filters.market_filter(
                event_type_ids=[horse_racing_id],  # filter on just horse racing
                market_countries=[""],  # filter on just GB countries
                market_type_codes=["WIN"],  # filter on just WIN market types
                venues = [sportsbet_venue]

            ),
            market_projection=[
                "MARKET_START_TIME",
                "RUNNER_DESCRIPTION",
            ],  # runner description required
            max_results=1,
        )



        print("%s market catalogues returned" % len(market_catalogues))

        if len(market_catalogues) == 0:
            print("No arbi op")

        else:
            for market_catalogue in market_catalogues:
                # prints market id, market name and market start time
                print(
                    market_catalogue.market_id,
                    market_catalogue.market_name,
                    market_catalogue.market_start_time,
                )

                print(f"https://www.betfair.com.au/exchange/plus/horse-racing/market/{market_catalogue.market_id}")
                for runner in market_catalogue.runners:
                    # prints runner id, runner name and handicap
                    print(runner.selection_id, runner.runner_name)

                # market book request
                market_books = trading.betting.list_market_book(
                    market_ids=[market_catalogue.market_id],
                    price_projection=filters.price_projection(
                        price_data=filters.price_data(ex_all_offers=True)
                    ),
                )

                for market_book in market_books:
                    # prints market id, inplay?, status and total matched
                    print(
                        market_book.market_id,
                        market_book.inplay,
                        market_book.status,
                        market_book.total_matched
                    )

                    market_id = market_book.market_id

                    if market_book.status == "OPEN":
                        for runner in market_book.runners:

                            #available_to_back = runner.ex.available_to_back[0]

                            try:
                                available_to_lay = {"horse_id" : runner.selection_id, "status" : runner.status, "size" : runner.ex.available_to_lay[0].size, "price" : runner.ex.available_to_lay[0].price}

                            except: 
                                available_to_lay = {"horse_id" : runner.selection_id, "status" : "No Lay"}

                            betfair_selections.append(available_to_lay)
                print("\n \n")

            for i in betfair_selections:
                print(i)

            print(len(selections))
            print(len(betfair_selections))

    ################################################################################



            for idx, i in enumerate(selections):
                    player_id = i["id"]
                    print(player_id)


                    try:
                        lay_price = betfair_selections[idx]["price"]
                        size = betfair_selections[idx]["size"]
                        print("Lay: ", lay_price, "Size: ", size)

                        try:
                            win_price = float(driver.find_element(By.XPATH, f'//div[@data-automation-id="racecard-outcome-{player_id}"]//div[@data-automation-id="racecard-outcome-0-L-price"]//span[@data-automation-id="price-text"]').text)
                            print("Win", win_price)

                    
                            if win_price >= lay_price*1.05:
                                        driver.find_element(By.XPATH, f'//div[@data-automation-id="racecard-outcome-{player_id}"]//span[@data-automation-id="price-text"]').click()
                                        print("Win:" + win_price + "Lay:" + lay_price)
                                        try:
                                                                            # Define a limit order filter
                                            limit_order_filter = betfairlightweight.filters.limit_order(
                                                size=5, 
                                                price=lay_price,
                                                persistence_type='LAPSE'
                                            )

                                            # Define an instructions filter
                                            instructions_filter = betfairlightweight.filters.place_instruction(
                                                selection_id=betfair_selections[idx]["horse_id"],
                                                order_type="LIMIT",
                                                side="LAY",
                                                limit_order=limit_order_filter
                                            )

                                            instructions_filter

                                            order = trading.betting.place_orders(
                                                market_id=market_id, # The market id we obtained from before
                                                instructions=[instructions_filter]) 
                                            
                                            order.__dict__
                                              
                                        except:
                                            print("Error with betfair order")

                        except:
                            print("Error with Back")

                    except:
                            print("Error with Back")

            #place_bet()

In [156]:
## Order output

{'orderType': 'LIMIT',
 'selectionId': 43016013,
 'side': 'LAY',
 'limitOrder': {'price': 1.65, 'persistenceType': 'LAPSE', 'size': 5}}

In [158]:
# order.__dict__ output

{'elapsed_time': 1.124047040939331,
 '_datetime_created': datetime.datetime(2022, 10, 11, 3, 33, 21, 447388),
 '_datetime_updated': datetime.datetime(2022, 10, 11, 3, 33, 21, 447388),
 '_data': {'status': 'SUCCESS',
  'marketId': '1.204693026',
  'instructionReports': [{'status': 'SUCCESS',
    'instruction': {'selectionId': 43016013,
     'limitOrder': {'size': 5.0, 'price': 1.65, 'persistenceType': 'LAPSE'},
     'orderType': 'LIMIT',
     'side': 'LAY'},
    'betId': '283818871867',
    'placedDate': '2022-10-11T03:33:18.000Z',
    'averagePriceMatched': 1.4395399999999998,
    'sizeMatched': 5.0,
    'orderStatus': 'EXECUTION_COMPLETE'}]},
 'market_id': '1.204693026',
 'status': 'SUCCESS',
 'customer_ref': None,
 'error_code': None,
 'place_instruction_reports': [<betfairlightweight.resources.bettingresources.PlaceOrderInstructionReports at 0x1ede22b9948>]}